# Chinook Database

We will explore the database "Chinook" that includes selling records of a record store. Let's check out the list of tables in our database first. We have left out our "in-depth" exploration of the data such as displaying tables for the sake of clarity.

To acces `chinook.db` [click here](https://github.com/lerocha/chinook-database)

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table



## 1. Adding new albums to the store
We need to find the most popular music genres in the USA to select the first albums we want to add to the store.

### 1.1 Number of tracks sold per genre in the USA
To do that we will need to use the following tables:
* `invoice_line`, to get all sales details
* `invoice` to get the country where the tracks were sold
* `track` to get the genre id
* `genre` to get the name of the genre

We will first create a view choosing tracks sold in the USA only. 
After that, we will associate a genre to each track sold, in order to count them.

In [3]:
%%sql

WITH
    usa_sales AS
        (
        SELECT il.*      
        FROM invoice i
        INNER JOIN invoice_line il ON il.invoice_id = i.invoice_id
        WHERE i.billing_country = 'USA'
        ),
    
    nb_tracks_genre AS
        (
        SELECT 
            g.name genre,
            COUNT(us.invoice_line_id) nb_tracks,
            ROUND((COUNT(t.track_id) / CAST((SELECT
                                      COUNT(*) 
                                  FROM usa_sales
                                 ) as FLOAT)) * 100,2) percentage
        FROM usa_sales us
        LEFT JOIN track t ON t.track_id = us.track_id
        LEFT JOIN genre g ON g.genre_id = t.genre_id
        GROUP BY 1
        )
               
SELECT 
    genre,
    nb_tracks,
    percentage
FROM nb_tracks_genre
ORDER BY 2 DESC;

Done.


genre,nb_tracks,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


### 1.2 Recommendation based on genre for the USA market

This first analysis shows that the "big family" of rock music is the most popular: Rock, Alt&Punk and Metal are the biggest genres sold on the store. This is also true when you notice that "Alternative" and "Heavy Metal" are listed a bit down the table, and could be aggregated to the Alternative & Punk and Metal genres.

Regarding the genre that interest us: Hip-Hop, Punk, Pop and Blues, Punk is the most popular bar far, followed by Blues.
Pop and Hip-Hop actually have similar numbers with a small advantage for Pop.

**To conclude, I would recommand the store to sell the "Red Tone" and "Slim Jim Bites" records. The other two albums may not be a good recommendation but if we had to choose one, technically "Meteor and the Girls", which is a pop album, could be the right choice.**

## 2. Customer and their sales rep study

### 2.1 Employee performance study
We will look at the performance for each employee (nb invoice, total $$, average basket, lowest basket, highest basket).
We will need to join the `employee`, `customer` and `invoice` tables.

In [4]:
%%sql
SELECT *
FROM employee;

Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
1,Adams,Andrew,General Manager,None,1962-02-18 00:00:00,2016-08-14 00:00:00,11120 Jasper Ave NW,Edmonton,AB,Canada,T5K 2N1,+1 (780) 428-9482,+1 (780) 428-3457,andrew@chinookcorp.com
2,Edwards,Nancy,Sales Manager,1,1958-12-08 00:00:00,2016-05-01 00:00:00,825 8 Ave SW,Calgary,AB,Canada,T2P 2T3,+1 (403) 262-3443,+1 (403) 262-3322,nancy@chinookcorp.com
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com
6,Mitchell,Michael,IT Manager,1,1973-07-01 00:00:00,2016-10-17 00:00:00,5827 Bowness Road NW,Calgary,AB,Canada,T3B 0C5,+1 (403) 246-9887,+1 (403) 246-9899,michael@chinookcorp.com
7,King,Robert,IT Staff,6,1970-05-29 00:00:00,2017-01-02 00:00:00,590 Columbia Boulevard West,Lethbridge,AB,Canada,T1K 5N8,+1 (403) 456-9986,+1 (403) 456-8485,robert@chinookcorp.com
8,Callahan,Laura,IT Staff,6,1968-01-09 00:00:00,2017-03-04 00:00:00,923 7 ST NW,Lethbridge,AB,Canada,T1H 1Y8,+1 (403) 467-3351,+1 (403) 467-8772,laura@chinookcorp.com


In [5]:
%%sql

SELECT
    e.first_name || ' ' || e.last_name AS name,
    e.hire_date,
    e.birthdate,
    e.country,
    COUNT(i.total) nb_invoices,
    ROUND(SUM(i.total),2) revenue,
    ROUND(AVG(i.total),2) avg_basket,
    ROUND(MAX(i.total),2) highest_basket,
    ROUND(MIN(i.total),2) lowest_basket
FROM invoice i
INNER JOIN customer c ON c.customer_id = i.customer_id
INNER JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY e.employee_id;

Done.


name,hire_date,birthdate,country,nb_invoices,revenue,avg_basket,highest_basket,lowest_basket
Jane Peacock,2017-04-01 00:00:00,1973-08-29 00:00:00,Canada,212,1731.51,8.17,23.76,0.99
Margaret Park,2017-05-03 00:00:00,1947-09-19 00:00:00,Canada,214,1584.0,7.4,19.8,0.99
Steve Johnson,2017-10-17 00:00:00,1965-03-03 00:00:00,Canada,188,1393.92,7.41,16.83,0.99


### 2.2 Conclusion
Out of the 8 employees, 3 are Sales Support Agents, all based in Canada. We can see that Jane Peacock and Margaret Park have somewhat similar results, while Steve Johnson seems to be a bit behind, with fewer customers and smaller baskets. 
Steve has been hired after the other 2 sales support agents, which could explain why he has completed fewer sales.

## 3 Sales data for customers from each different country
Let's count the number of customers per country. 
First subquery adds a new column where the countries with one customer are named 'Others'. 
Second subquery counts what we need for each country.
Third subquery creates a way to sort the 'Others' at the bottom.
The main query calls the country from the first subquery, the calculations from the second, and groups & sorts using the last subquery.

In [6]:
%%sql

WITH 
    country_count AS
        (
        SELECT 
            country,
            COUNT(customer_id) count,
            CASE
                WHEN COUNT(country) = 1 THEN 'Others'
                ELSE country
            END as sort               
        FROM customer
        GROUP BY country
        ORDER BY count DESC
        ),
    country_sales AS
        (
        SELECT 
            c.country country,
            COUNT(i.invoice_id) count_sales,
            ROUND(SUM(i.total),2) total_sales,
            ROUND(AVG(i.total),2) avg_sales,
            ROUND((SUM(i.total) / COUNT(i.invoice_id)),2) avg_customer
        FROM customer c
        LEFT JOIN invoice i ON i.customer_id = c.customer_id
        GROUP BY c.country
        ),
    sorted_country AS
        (
        SELECT
            sort as sorting_country,
            SUM(count) customers,
            CASE
                WHEN sort = 'Others' THEN 1
                ELSE 0
            END sorting
        FROM country_count
        GROUP BY sorting_country
        )
        
SELECT 
    cc.sort country,
    sc.customers customers,
    cs.total_sales total_sales,
    cs.avg_sales avg_sales,
    cs.avg_customer avg_basket
FROM country_count cc
LEFT JOIN sorted_country sc ON cc.sort = sc.sorting_country
LEFT JOIN country_sales cs ON cs.country = cc.country
GROUP BY cc.sort
ORDER BY sc.sorting,customers DESC
;

Done.


country,customers,total_sales,avg_sales,avg_basket
USA,13,1040.49,7.94,7.94
Canada,8,535.59,7.05,7.05
Brazil,5,427.68,7.01,7.01
France,5,389.07,7.78,7.78
Germany,4,334.62,8.16,8.16
United Kingdom,3,245.52,8.77,8.77
Czech Republic,2,273.24,9.11,9.11
India,2,183.15,8.72,8.72
Portugal,2,185.13,6.38,6.38
Others,15,75.24,7.52,7.52


## 4. Full albums vs. Tracks

### 4.1 Full album sales
We will count the number of tracks for each albums and confront that data to the sales data. We will be able to figure out the number of invoices that include a full album sale.

* number of invoices
* percentage of invoices



In [15]:
%%sql

WITH
    invoices AS
        (
        SELECT
            il.invoice_id invoice_id,
            t.album_id album_id,
            COUNT(il.track_id) nb_tracks
        FROM invoice_line il
        LEFT JOIN track t ON t.track_id = il.track_id
        GROUP BY il.invoice_id, t.album_id
        ),
    album_tracks AS
        (
        SELECT
            album_id,
            COUNT(track_id) AS count
        FROM track
        GROUP BY album_id
        ),
    album_invoice AS
        (
        SELECT 
            i.invoice_id,
            CASE
                WHEN i.nb_tracks = at.count THEN 1
                ELSE 0
            END full_album
        FROM invoices i
        LEFT JOIN album_tracks at ON at.album_id = i.album_id
        WHERE i.nb_tracks > 1
        ),
    invoice_counts AS
        (
        SELECT
            invoice_id,
            COUNT(full_album)
        FROM album_invoice ai
        WHERE full_album >=1
        GROUP BY invoice_id
        )
SELECT
    COUNT(invoice_id) nb_invoices,
    ROUND((CAST(COUNT(invoice_id) as FLOAT) / (SELECT COUNT(*)
                          FROM invoice)) * 100,2) || '%' AS percentage
FROM invoice_counts il
;
        



Done.


nb_invoices,percentage
111,18.08%


### 4.2 Conclusion
There are 18% of the invoices that include at least a full album sale. This excludes 1-track bundles which we know are usually singles released before album announcement.
The store should definitely continue buying full albums, however they might want to dig into this further to see if there is a specific genre that sells more full albums than others.